# Imports

In [1]:
import os
import pandas as pd
import numpy as np
import pickle

from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer

from tpot import TPOTRegressor
from tempfile import mkdtemp
from shutil import rmtree

ImportError: cannot import name 'SCORERS' from 'sklearn.metrics' (/home/tatchiwiggers/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/metrics/__init__.py)

In [ ]:
#setting the display for Pipelines and ColumnTransformers
from sklearn import set_config
set_config(display='diagram')

# Pipelines

🎯 We are going to predict the charges of a health insurance contract based on various features using the following dataset.

💾 Download the dataset [here](https://wagon-public-datasets.s3.amazonaws.com/data_workflow.csv)

In [ ]:
data = pd.read_csv('data/data_workflow.csv')
data.head()

In [ ]:
data.shape

Now let's define our `X` and out `y`, so that we have something to work with. We will also do a quick manual Holdout Method.

In [ ]:
X = data.drop(columns='charges')
y = data['charges']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

Today we will: <br>
1. Impute missing values
2. Scale numerical features
3. Encode categorical features
4. Fine-tune our model and our preprocessing

All at once, in one single cell!! :D 

## Preprocessing Pipelines

### Pipeline

In [ ]:
data.head()

In [ ]:
# from sklearn.pipeline import Pipeline

# Preprocess the "age" column
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('standard_scaler', StandardScaler())
])

pipeline.fit(X_train[['age']])
pipeline.transform(X_train[['age']])

We can now look at our pipeline, as well as access individual steps of it:

In [ ]:
pipeline

In [ ]:
print(pipeline[1]) # by index
print(pipeline['standard_scaler']) # by name

### Column Transformer ⑂

Column Transformers allow you to apply specific changes to specific columns in parallel

For this one we will:<br>
    1. Impute and scale numerical values<br>
    2. Encode categorical values

In [ ]:
# from sklearn.compose import ColumnTransformer

# Impute then scale numerical values: 
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy="mean")),
    ('standard_scaler', StandardScaler())
])

# Encode categorical values
cat_transformer = OneHotEncoder(handle_unknown='ignore')

# Parallelize "num_transformer" and "cat_transfomer"
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['age', 'bmi']),
    ('cat_transformer', cat_transformer, ['smoker', 'region'])
])

Let's checkout out our CT!

In [ ]:
preprocessor

That's cool but how do we use it?

In [ ]:
X_train_transformed = preprocessor.fit_transform(X_train)

#Using display instead of print to see a proper DF 
# in the form of a table with borders around rows and columns
print("Original training set")
display(X_train.head(3))

print("Preprocessed training set - Notice how we are missing feature names!")
display(pd.DataFrame(data=X_train_transformed).head(3))

In [ ]:
preprocessor.get_feature_names_out()

In [ ]:
pd.DataFrame(
    data = X_train_transformed,
    columns=preprocessor.get_feature_names_out()
).head()

what about the children? We can't leave them behind! :(

In [ ]:
#Making sure the children are safe and sound
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['age','bmi']),
    ('cat_transformer', cat_transformer, ['region','smoker'])],
    remainder='passthrough'
)

preprocessor

In [ ]:
pd.DataFrame(
    preprocessor.fit_transform(X_train),
    columns=preprocessor.get_feature_names_out()
).head(3)

### Custom: Function Transformer →

If you'd like to do you r own thing, you can use custom transformers. These encapsulate a Python function into a transformer object. As it is a transformer, it still works with Pipelines and other transformers, like the ColumnTransformer we just saw.

In [ ]:
# from sklearn.preprocessing import FunctionTransformer

# Create a transformer that compresses data to 2 digits (for instance!)
# rounder = FunctionTransformer(np.round)

# We can use a lambda function for more customizable functions
rounder = FunctionTransformer(lambda array: np.round(array, decimals=2))

In [ ]:
# Add it at the end of our numerical transformer
num_transformer = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('rounder', rounder)])

# Encode categorical values
cat_transformer = OneHotEncoder(drop='if_binary',
                                handle_unknown='ignore')

preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['bmi', 'age']),
    ('cat_transformer', cat_transformer, ['region', 'smoker'])],
    remainder='passthrough')
preprocessor

In [ ]:
pd.DataFrame(preprocessor.fit_transform(X_train)).head(3)

### Feature Union ||

We can take this a step further! By applying entire transformers in parallel.<br>
yes we can! Let's do that and create a nwe feature as well: `bmi_age_ratio`.

In [ ]:
X_train.head(3)

In [ ]:
# from sklearn.pipeline import FeatureUnion

# Create a custom transformer that multiplies/divides two columns
# Notice that we are creating this new feature completely randomly just as an example
bmi_age_ratio_constructor = FunctionTransformer(lambda df: pd.DataFrame(df["bmi"] / df["age"]))

union = FeatureUnion([
    ('preprocess', preprocessor), # columns 0-7
    ('bmi_age_ratio', bmi_age_ratio_constructor) # new column 8
])

union

### Summary with make_*** shortcuts

In [ ]:
# from sklearn.pipeline import make_pipeline
# from sklearn.pipeline import make_union
# from sklearn.compose import make_column_transformer

Pipeline([
    ('my_name_for_the_imputer', SimpleImputer()),
    ('my_name_for_the_scaler', StandardScaler())
])

In [ ]:
make_pipeline(SimpleImputer(), StandardScaler()) # here we dont get to name our steps

Let's try and redo it all with this new method!

In [ ]:
num_transformer = make_pipeline(SimpleImputer(), StandardScaler())
cat_transformer = OneHotEncoder()

preproc_basic = make_column_transformer(
    (num_transformer, ['age', 'bmi']),
    (cat_transformer, ['smoker', 'region']),
    remainder='passthrough'
)

preproc_full = make_union(preproc_basic, bmi_age_ratio_constructor)
preproc_full

Cool right? The upside is that it's much faster to write and implement, but the big downside is that you can't select you own names for transformers, which may be something that you care about doing.<br>
We can also use `make_column_selector`to select features to be used based on their `dtype`.

In [ ]:
# from sklearn.compose import make_column_selector

num_col = make_column_selector(dtype_include=['float64'])
cat_col = make_column_selector(dtype_include=['object','bool'])

X_train.dtypes

### Complete Preprocessing Pipeline

In [ ]:
# from sklearn.compose import make_column_selector

#IMpute then scale numerical values
num_transformer = make_pipeline(SimpleImputer(), StandardScaler())
num_col = make_column_selector(dtype_include=['float64'])

# Encode categorical values
cat_transformer = OneHotEncoder()
cat_col = make_column_selector(dtype_include=['object','bool'])

# Use the ColumnTransformer to parallelize these operations
preproc_basic = make_column_transformer(
    (num_transformer, num_col),
    (cat_transformer, cat_col),
    remainder='passthrough'
)

preproc_full = make_union(preproc_basic, bmi_age_ratio_constructor)
preproc_full

## Including Models in Pipelines

In [ ]:
# from sklearn.linear_model import Ridge

# Preprocessor -Impute then scala numerical values
num_transformer = make_pipeline(SimpleImputer(), StandardScaler())
# encode cateforical values
cat_transformer = OneHotEncoder()

# use the ColumnTransformer to parallelize these operations
preproc = make_column_transformer(
    (num_transformer, make_column_selector(dtype_include=['float64'])),
    (cat_transformer, make_column_selector(dtype_include=['object','bool'])),
    remainder='passthrough'
)

# Add estimator
pipeline = make_pipeline(preproc, Ridge())
pipeline

Because my pipeline inherits from my last step, i can do this:

In [ ]:
# Train Pipeline
pipeline.fit(X_train,y_train)

# Make predictions
pipeline.predict(X_test.iloc[0:1])

# Score model
pipeline.score(X_test,y_test)

### Cross-validate Pipeline

In [ ]:
# from sklearn.model_selection import cross_val_score

cross_val_score(pipeline, X_train, y_train, cv=5, scoring='r2').mean()

### Gridsearch a Pipeline

Believe it.. Grid Search is also possible!

In [ ]:
# Which parameters of the pipeline are GridSearch-able? here you can look for the params you wanna grid search
# pipeline.get_params()

In [ ]:
# from sklearn.model_selection import GridSearchCV

# instantiate a GS
grid_search = GridSearchCV(
    pipeline, # instead of our model
    param_grid={
        # we can access ANY component of the Pipeline
        # and ANY available hyperparamater you want to optimize - These are double
        # underscores also referred to as "dunder"
        #step              #pipeline  #transformer   #strat
        'columntransformer__pipeline__simpleimputer__strategy': ['mean', 'median'],
        'ridge__alpha': [0.1, 0.5, 1, 5, 10]
    },
    cv=5,
    scoring="r2")

grid_search.fit(X_train, y_train)
grid_search.best_params_

💾 Let's save the pipelined model with the best hyperparameters.

In [ ]:
pipeline_tuned = grid_search.best_estimator_
pipeline_tuned

🔮 We can use this "best" model for predictions without re-training it!

In [ ]:
pipeline_tuned.predict(X_test[0:1])

An extra tip for you guys:

### Cache to avoid repeated computations

Are your preprocessing steps too long to run?

You can use caching techniques!

In [ ]:
# from tempfile import mkdtemp
# from shutil import rmtree

# Create a temp folder
cache_dir = mkdtemp()

# Instantiate the Pipeline with the cache parameter
pipeline = make_pipeline(preproc_basic, Ridge(), memory=cache_dir)

#instantiate a GS
# instantiate a GS
grid_search = GridSearchCV(
    pipeline, 
    param_grid={
        # Access any component of the Pipeline
        # and ANY available hyperparamater you want to optimize - These are double
        # underscores also referred to as "dunder"
        'columntransformer__pipeline__simpleimputer__strategy': ['mean', 'median'],
        'ridge__alpha': [0.1, 0.5, 1, 5, 10]
    },
    cv=5,
    scoring="r2")
# Clear the cache directory after the cross-validation
rmtree(cache_dir)

In [ ]:
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
# Clear the directory after the CV
rmtree(cache_dir)

###  Debug your pipe

In [ ]:
# Access the components of a Pipeline with `named_steps`
pipeline_tuned.named_steps.keys()

In [ ]:
# Check intermediate steps
print("Before preprocessing, X_train.shape = ")
print(X_train.shape)
print("After preprocessing, X_train_preprocessed.shape = ")
pipeline_tuned.named_steps["columntransformer"].fit_transform(X_train).shape

### Exporting models/Pipelines

💾 You can export your final model/pipeline as a pickle file so you can load them into your python file or your notebook so you can use it directly as it is.

👉The file can then be loaded back into a notebook or deployed on a server (see ML Ops module).

In [ ]:
# import pickle

# Export Pipeline as pickle file
with open("pipeline.pkl", "wb") as file:
    pickle.dump(pipeline_tuned, file)

# Load Pipeline from pickle file
my_pipeline = pickle.load(open("pipeline.pkl","rb"))

my_pipeline.score(X_test, y_test)

# That cool thing I told you about 🎉

**Auto ML**

So here we have our training and testing sets.

In [ ]:
# import os
# from tpot import TPOTRegressor

X_train_preproc = preproc_basic.fit_transform(X_train)
X_test_preproc = preproc_basic.transform(X_test)

In [ ]:
# Instantiate TPOTClassifier
                    #like epochs    #how much info
tpot = TPOTRegressor(generations=4, population_size=20, verbosity=2, scoring='r2', n_jobs=-1, cv=2)

# Process autoML with TPOT
tpot.fit(X_train_preproc, y_train)

# Print score
print(tpot.score(X_test_preproc, y_test))


In [ ]:
# Export TPOT Pipeline to a Python file
tpot.export(os.path.join(os.getcwd(),'tpot_iris_pipeline.py'))

! cat 'tpot_iris_pipeline.py'